---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
#import re
def date_sorter():
     # doc = []
    # df_dates = df.str.extractall(r'((\d?\d[/-]){1,2}(\d\d\d?\d?))|(\d\d\d\d){4}')
    #df_dates = df.str.extractall(r'(((\d?\d[/-]){1,2}(\d\d|\d\d\d\d))|(20\d\d)|(19\d\d)|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.\d\d\d?\d?))')
    #df_dates = df.str.extractall(r'((20\d\d)|(19\d\d)|(\d?\d/\d\d\d\d)|((\d?\d[/-]){1,2}(\d\d|\d\d\d\d))|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.\d\d\d?\d?))')
    pattern1 = r'((\d\d?[,\s\.-]{1,2}(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,\s\.-]{1,2}\d\d\d?\d?)|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,\s\.-]{1,2}\d{2}[,\s\.-]{1,2}\d\d\d?\d?)|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,\s\.-]{1,2}\d\d\d?\d?))'
    pattern2 = r'(((?:\d\d?[/]){1,2}\d\d\d?\d?)|((?:\d\d?[-]){2}\d\d\d?\d?))'
    pattern3 = r'((19\d\d)|(20\d\d))'


    #df_dates = df.str.extractall(r'((20\d\d)|(19\d\d)|(\d?\d/\d\d\d\d)|((\d?\d[/-]){1,2}(\d\d|\d\d\d\d))|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.\d\d\d?\d?))')
    df_dates1 = df.str.extractall(pattern1).reset_index()
    df_dates2 = df.str.extractall(pattern2).reset_index()
    df_dates3 = df.str.extractall(pattern3).reset_index()

    #print(df_dates1)
    #print(df_dates1.columns)
    df_dates1 = df_dates1[['level_0', 0]]
    df_dates2 = df_dates2[['level_0', 0]]
    df_dates3 = df_dates3[['level_0', 0]]

    df_dates1_2 = pd.merge(df_dates1,  df_dates2, on='level_0', how = 'outer')
    df_dates = pd.merge(df_dates1_2, df_dates3, on='level_0', how = 'outer')
    #print(df_dates)

    #df_dates = df_dates[5:15]
    #df_dates = df.str.extractall(r'((20\d\d)|(19\d\d)|(\d?\d/\d\d\d\d)|((\d?\d[/-]){1,2}(19|20\d\d))|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.\d\d\d?\d?))')
    df_dates_grouped = df_dates.drop_duplicates(subset='level_0', keep='first') #df_dates[~df_dates.index.duplicated(keep='last')] # df_dates.groupby(level=df_dates.index.names).last()#df_dates.reset_index().drop_duplicates(subset='index', keep='first')  #.set_index('index')
    df_dates_grouped.columns = ['level_0', 'a', 'b', 'c']
    df_dates_grouped['a'] #.str.strip()
    df_dates_grouped['b'] #.str.strip()
    df_dates_grouped['c'] #.str.strip()
    #df_dates_grouped['date'] = df_dates_grouped.apply(lambda x: x['c'] if (x['a'] == None) and (x['b']== None) else (x['b'] if x['a'] == None else x['a'] ), axis=1)
    df_dates_grouped['date'] = df_dates_grouped.apply( lambda x: x['a'] if (x['a'] == x['a']) else (x['b'] if x['b'] == x['b'] else x['c']), axis=1)
    #df_dates_grouped['date'] = df_dates_grouped.apply(lambda x: x['c'] if ((x['a'] == x['a']) and (x['b'] == x['b']))  else x['a'], axis=1)
    # df_dates1 = df.str.extractall(r'((\d?\d[/-]){1,2}(\d\d\d?\d?))|(20\d\d)|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).\d\d\d?\d?)')
    # df_dates2 = df.str.extractall(r'((\d?\d[/-]){1,2}(\d\d\d?\d?))|(20\d\d)|(\d\d.(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).\d\d\d?\d?)')
    df_dates_grouped = df_dates_grouped[['level_0', 'date']]
    d_months = {'Jan' : 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    # # extract year
    # df_dates_grouped['dateformatted'] =  df_dates_grouped[0].str.findall(r'(20\d\d|19\d\d)')
    # df_dates_grouped['dateformatted2'] = df_dates_grouped[0].str.findall(r'[-/\s\.]*20\d\d|19\d\d')
    # df_dates_grouped['dateformatted3'] = df_dates_grouped[0].str.findall(r'(\d?\d[-/\.]){1,2}(\d\d)')
    # df_dates_grouped['dateformatted4'] = df_dates_grouped[0].str.split(r'[-/\.\s]')
    df_dates_grouped[['day', 'month', 'year']] = df_dates_grouped['date'].str.split(r'[\s/-]',expand=True)   #df_dates_grouped['dateformatted4']
    #
    df_dates_grouped['year2'] =  df_dates_grouped.apply(lambda x: x['day'] if (x['year'] == None) and (x['month'] == None) else (x['month'] if x['year'] == None else x['year'] ), axis=1)
    df_dates_grouped['month2']  = df_dates_grouped.apply(lambda x: '01' if (x['month'] == None) and (x['year'] == None) else (x['day'] if x['year'] == None else x['month'] ), axis=1)
    df_dates_grouped['day2'] = df_dates_grouped.apply(lambda x: '01' if x['year'] == None else x['day'], axis=1)
    #
    df_dates_grouped['year3'] = df_dates_grouped['year2'].apply(lambda x: '19' + x if len(x) == 2 else x)
    #df_dates_grouped['month3'] =  df_dates_grouped.apply(lambda x: x['month2'] if (re.search(r'[a-zA-Z]', x['month2']) or x['year'] == None) else x['day2'], axis=1)
    df_dates_grouped['month3'] =  df_dates_grouped.apply(lambda x: x['month2'] if ((x['month2'][0].isalpha()) or x['year'] == None) else x['day2'], axis=1)
    #df_dates_grouped['month_letter'] = df_dates_grouped.apply(lambda x: x['month2'][0], axis=1)
    #df_dates_grouped['month_letter_match'] = df_dates_grouped.apply(lambda x: x['month2'][0].isalpha(), axis=1)
    #
    #df_dates_grouped['day3'] = df_dates_grouped.apply(lambda x: x['month2'] if re.search(r'[0-9]', x['month2']) else x['day2'], axis=1)
    df_dates_grouped['day3'] = df_dates_grouped.apply(lambda x: x['month2'] if x['month2'][0].isdigit() else x['day2'], axis=1)
    df_dates_grouped['day3'] = df_dates_grouped.apply(lambda x: '1' if (x['year'] == None and x['month'] != None and x['day'] != None) else x['day3'], axis=1)
    df_dates_grouped['month4'] = df_dates_grouped['month3'].apply(lambda x: d_months.get(x[:3], x))
    df_dates_grouped['day3'] = df_dates_grouped['day3'].str.replace(r'[^0-9]', '')
    df_dates_grouped[['yr', 'mos', 'dt']] = df_dates_grouped[['year3', 'month4', 'day3']].apply(pd.to_numeric)
    #df_dates_grouped.sort_values(['year3', 'month4', 'day3'], ascending=['True','True','True'], inplace = True)
    df_dates_grouped.sort_values(['yr', 'mos', 'dt'], ascending=['True', 'True', 'True'], inplace=True)
    df_dates_grouped = df_dates_grouped.reset_index()
    #print(df_dates_grouped.index)
    return df_dates_grouped['level_0']  #['year'] #.set_index('level_0')[0]#df_dates.reset_index() #df_dates_grouped.shape#df_dates_grouped#df_dates #[0]  # .loc[120:130]# Your answer here
   

#date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64